Let's start by creating a [Knowledge Base for Amazon Bedrock](https://aws.amazon.com/bedrock/knowledge-bases/) 
to provide knowledge about mortgages. In this notebook, we will create a knowledge base using the content available in the mortgage_dataset folder. This knowledge base will be used to answer general QnA.  As a challenge, see if you can include more data sources 

Step 1: Import libraries and helper functions

In [ ]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent

%load_ext autoreload
%autoreload 2

In the following cell, we add `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path.

These file contain helper classes focused on making labs experience smooth. 

All interactions with Bedrock will be handled by these classes.

Following methods will be used in this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys
sys.path.insert(0, '..') 

from src.utils.bedrock_agent import AgentsForAmazonBedrock

from src.utils.knowledge_base_helper import KnowledgeBasesForAmazonBedrock

agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

Create boto3 clients

In [ ]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Get the region and bucket name. The bucket will be created if its not present already.

In [ ]:
region = boto3.session.Session().region_name
account_id = sts_client.get_caller_identity()["Account"]
suffix = f"{region}-{account_id}"
bucket_name = f'mac-workshop-{suffix}'

In [ ]:
agent_foundation_model = ["anthropic.claude-3-5-haiku-20241022-v1:0"]

### Create Knowledge Base 
 We will now create the knowledge base with Amazon OpenSearch Serverless as the vector store. To do so, we will use the helper class `KnowledgeBasesForAmazonBedrock` which creates the knowledge base and all of its prerequisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge Base
7. Knowledge Base data source

This might take a few minutes, so have a break!

In [ ]:
knowledge_base_name = "mortgage-agent-kb"

knowledge_base_description = "KB containing information on mortgages"

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")


Now that we have created a knowledge base, let's populate it with some mortgage knowledge. For this workshop we have two comprehensive articles from Investopedia.com for illustrative purposes.

The Knowledge Base data source expects the data to be available on the S3 bucket connected to it and changes on the data can be syncronized to the knowledge base using the Ingest API call.  Lets upload the files to the s3 bucket

In [ ]:
# function to upload to S3 bucket
import boto3

def upload_file_to_s3(file_path, bucket_name, object_key=None):
    """Upload a file to S3 bucket"""
    s3_client = boto3.client('s3')
    
    # Check if bucket exists, create if not
    existing_buckets = [bucket['Name'] for bucket in s3_client.list_buckets()['Buckets']]
    if bucket_name not in existing_buckets:
        s3_client.create_bucket(Bucket=bucket_name)
    
    if object_key is None:
        object_key = file_path.split('/')[-1]
    
    s3_client.upload_file(file_path, bucket_name, object_key)
    return f"s3://{bucket_name}/{object_key}"


In [ ]:
upload_file_to_s3("mortgage_dataset/15-Year vs. 30-Year Mortgage What's the Difference .html", bucket_name,"15-Year vs. 30-Year Mortgage What's the Difference .html")

In [ ]:
upload_file_to_s3("mortgage_dataset/Mortgage Refinancing When Does It Make Sense .html", bucket_name,"Mortgage Refinancing When Does It Make Sense .html")

Now we ingest the documents, which chunks the source documents and stores an embedding for each chunk into the underying knowledge base vector store. For a simple example, this ingestion takes a couple minutes.

In [ ]:
%%time
# Start an ingestion job to synchronize data
kb.synchronize_data(kb_id, ds_id)
print('KB synchronization completed\n')

### Test the Knowledge Base
Now the Knowledge Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a Bedrock LLM.

In [ ]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "compare and contrast 15-year vs 30-year mortgage type"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model[0]}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

As you can see, with the retrieve and generate API we get the final response directly and we don't see the different sources used to generate this response. Let's now retrieve the source information from the knowledge base with the retrieve API.

**Testing Knowledge Base with Retrieve API**

If you need an extra layer of control, you can retrieve the chuncks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata.

In [ ]:
 response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":3,
        } 
    },
    retrievalQuery={
        'text': 'What are the cons of a 15-year mortgage?'
    }
)

def response_print(retrieve_resp):
    #structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print('-----------------------------------------------------------------------------------------')
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

Store knowledge base ID and name for subsequent labs

In [ ]:
%store kb_id
%store knowledge_base_name

In this lab, we created a knowledge base that will be used by one of the agents to answers queries on mortgage. In the next lab, we will create two more agents that can achieve specific tasks. 